# Initializing Dash for visualization

## imports

In [1]:
import os
import sys
sys.path.insert(0,'../..')

import pandas as pd
from PIL import Image
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from hockey.src.data.team_data import  get_all_teams


teams_lst = get_all_teams(data_dir="../data/raw")
years = [2016, 2017, 2018, 2019, 2020]

## Adding the graph, slider and two dropdown lists 
for the NHL rink, year, team and season respectively

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.Div([
        "Team: ",
        dcc.Dropdown(
                    id='teams-dropdown',
                    options=[{'label': i, 'value': i} for i in teams_lst],
                    value=teams_lst[0]
                ),
        "Season: ",
        dcc.Dropdown(
                    id='seasons-dropdown',
                    options=[{'label': i, 'value': i} for i in ['regular', 'postseason']],
                    value='regular'
                ),
        ],
        style={'width': '49%', 'display': 'inline-block'}),
    
    html.Div([
        dcc.Graph(id='nhl-rink-graph'),
        dcc.Slider(
            id='year-slider',
            min=years[0],
            max=years[-1],
            value=years[0],
            marks={str(year): str(year) for year in years},
            step=None
        )
    ])
])

## Visualizing the shots for a given team, year and season

In [3]:
teams_data_dir = "../data/teams"

In [4]:
def flip_coord_to_one_side(team_df):
    for idx, event_row in team_df.iterrows():
        if event_row['coordinate_x'] < 0:
            team_df.at[idx,'coordinate_x']= event_row['coordinate_x'] * -1
            team_df.at[idx,'coordinate_y']= event_row['coordinate_y'] * -1

In [5]:
@app.callback(
    Output('nhl-rink-graph', 'figure'),
    [Input('year-slider', 'value'),
     Input('teams-dropdown', 'value'),
     Input('seasons-dropdown', 'value')])
def update_figure(selected_year, selected_team, selected_season):
    img = Image.open('../figures/nhl_rink.png')
    
    # get the dataframe of the chosen team
    team_df = pd.read_csv(os.path.join(teams_data_dir, selected_team+'.csv'))
    
    # and the chosen year
    team_df = team_df.loc[team_df['year'] == selected_year]
    
    # and the chosen season
    team_df = team_df.loc[team_df['season'] == selected_season]
    
    # Flip all the coordinates to the same side
    flip_coord_to_one_side(team_df)
    
    fig = px.scatter(team_df, x="coordinate_x", y="coordinate_y", color="shooter_name") #,
                     #size="pop", color="continent", hover_name="country")

    
    # set the axes
    fig.update_yaxes(range=[-45, 45], 
                     scaleratio = 1, 
                     scaleanchor="y",
                     showgrid=True,
                     zeroline=True,
                     showline=True,
                     gridcolor='#bdbdbd',
                     gridwidth=2,
                     zerolinecolor='#969696',
                     zerolinewidth=4,
                     linecolor='#636363',
                     linewidth=6)
    fig.update_xaxes(range=[-100, 100], 
                     scaleratio = 1, 
                     scaleanchor="x",
                     showgrid=True,
                     zeroline=True,
                     showline=True,
                     gridcolor='#bdbdbd',
                     gridwidth=2,
                     zerolinecolor='#969696',
                     zerolinewidth=4,
                     linecolor='#636363',
                     linewidth=6)

    # remove any margins and padding
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        margin=dict(l=0, #left margin
                    r=0, #right margin
                    b=0, #bottom margin
                    t=0,  #top margin
                    pad=0
                   )
    )


    # Add the NHL rink image
    fig.add_layout_image(
            dict(
                source=img,
                xref="x",
                yref="y",
                x=0,
                y=0,
                sizex=200,
                sizey=85,
                sizing= "stretch",
                layer = "below",
                xanchor = "center",
                yanchor = "middle",
                opacity = 0.8),
    )

    return fig    

app.run_server()

Dash app running on http://127.0.0.1:8050/
